In [ ]:
import ee
import folium
import numpy as np
import time
import random
import geemap
import geopandas as gpd

import sys
sys.path.insert(0, "../../src")

from etl import *
from viz import *

In [ ]:
ee.Authenticate()

In [ ]:
ee.Initialize()

In [ ]:
# Load bbox+fire geometries and push to EE as FeatureCollection
bbox_df = gpd.read_file("../../data/unburned/bbox.shp")
fireBounds_df = gpd.read_file("../../data/burned/fireBounds.shp")

bbox_EE = geemap.gdf_to_ee(bbox_df, geodesic=True)
fireBounds_EE = geemap.gdf_to_ee(fireBounds_df, geodesic=True)

In [ ]:
# can check if data is successfully pushed to EE
# print(bbox_EE.size().getInfo(),
#       fireBounds_EE.size().getInfo())

In [ ]:
firePts, bboxPts = ee.List([]), ee.List([])

for name, size in fireBounds_df[["FIRE_NAME", "GIS_ACRES"]].values:
    if size < 40000:
        gridScale = 80
    elif size < 90000:
        gridScale = 100
    elif size < 150000:
        gridScale = 110
    else:
        gridScale = 125

    firePts = firePts.add(genSamplePoints(feature=fireBounds_EE.filter(ee.Filter.eq("FIRE_NAME", name)),
                                          gridScale=gridScale-15,
                                          pointScale=1/8,
                                          seed=random.randint(0, 1e6)))
    
    bboxPts = bboxPts.add(genSamplePoints(feature=bbox_EE.filter(ee.Filter.eq("FIRE_NAME", name)),
                                          gridScale=gridScale+10,
                                          pointScale=1/8,
                                          seed=random.randint(0, 1e6)))

In [ ]:
startTime = time.time()

firePts_df = formatToGPD(list(fireBounds_df["FIRE_NAME"]), firePts.getInfo())   # 167k points
bboxPts_df = formatToGPD(list(fireBounds_df["FIRE_NAME"]), bboxPts.getInfo())   # 165k points

print("Point Sampling Runtime: {} minutes".format(np.round((time.time()-startTime)/60, 3)))

In [ ]:
bounds_df = gpd.read_file("../../data/bounds/bounds.shp")

In [1]:
# Reduce data over sample points in EE
startTime = time.time()
imageLst = ee.List([])
fireSampleData, bboxSampleData = [], []

for fireName, preFireDate, postFireDate, geometry in bounds_df[["FIRE_NAME", "pre-date", "post-date", "geometry"]].values[11:]:
    t1 = time.time()

    firePts_EE = geemap.gdf_to_ee(firePts_df[firePts_df["FIRE_NAME"]==fireName])
    bboxPts_EE = geemap.gdf_to_ee(bboxPts_df[bboxPts_df["FIRE_NAME"]==fireName])
    points = ee.List([firePts_EE, bboxPts_EE])

    geometry = ee.Geometry.Rectangle(list(geometry.bounds))

    # Get pre-post fire Landsat 8 images
    preFireImage_l8 = mosaicByDate(ee.ImageCollection("LANDSAT/LC08/C02/T1_L2"
                                    ).filterBounds(geometry
                                    ).filterDate(preFireDate,
                                                 ee.Date(preFireDate).advance(1, "day")))

    postFireImage_l8 = mosaicByDate(ee.ImageCollection("LANDSAT/LC08/C02/T1_L2"
                                     ).filterBounds(geometry
                                     ).filterDate(postFireDate,
                                                  ee.Date(postFireDate).advance(1, "day")))

    preFireImage_l8, postFireImage_l8 = ee.Image(preFireImage_l8.get(0)), ee.Image(postFireImage_l8.get(0))

    # Calculate NBR, dNBR, and burn severity
    preFireNBR = preFireImage_l8.normalizedDifference(['SR_B5', 'SR_B7'])
    postFireNBR = postFireImage_l8.normalizedDifference(['SR_B5', 'SR_B7'])
    dNBR = (preFireNBR.subtract(postFireNBR)).multiply(1000).rename("dNBR")

    burnSeverity = dNBR.expression(" (b('dNBR') > 425) ? 4 "    # purple: high severity
                                   ":(b('dNBR') > 225) ? 3 "    # orange: moderate severity
                                   ":(b('dNBR') > 100) ? 2 "    # yellow: low severity
                                   ":(b('dNBR') > -60) ? 1 "    # green: unburned/unchanged
                                   ":0"                         # brown: vegetation growth
                      ).rename("burnSeverity")
    
    # Get SRTM elevation, NLCD land cover, NDVI, and GRIDMET weather
    dem = ee.Image("NASA/NASADEM_HGT/001").select("elevation")
    nlcd2016 = ee.ImageCollection('USGS/NLCD_RELEASES/2016_REL'
                ).filter(ee.Filter.eq('system:index', '2016')).first()
    
    ndvi = postFireImage_l8.normalizedDifference(["SR_B5", "SR_B4"]).rename("NDVI")
    
    gridmet = ee.ImageCollection("IDAHO_EPSCOR/GRIDMET"
               ).filterBounds(geometry
               ).filterDate(ee.Date(postFireDate).advance(-3, "day"), postFireDate
               ).mean()

    # Merge all image bands together
    combined = postFireImage_l8.select('SR_B.'          # post-fire L8 bands 1-7
                              ).addBands(burnSeverity   # classified burn severity
                              ).addBands(dNBR           # dNBR
                              ).addBands(ndvi           # post-fire NDVI
                              ).addBands(dem            # SRTM elevation
                              ).addBands(gridmet        # all GRIDMET bands
                              ).addBands(nlcd2016       # all NLCD bands
                              ).set("FIRE_NAME", fireName)

    imageLst = imageLst.add(combined)

    # apply reducer and save results
    reducedPts = points.map(lambda x: pointReducer(image=combined,
                                                   collection=x,
                                                   scale=30,
                                                   reducer=ee.Reducer.mean()))
    lst_1, lst_2 = reducedPts.getInfo()

    fireSampleData.append(lst_1)
    bboxSampleData.append(lst_2)
    print("{} Runtime: {} minutes".format(fireName, np.round((time.time()-t1)/60, 3)))
    
print("Total Runtime: {} minutes".format(np.round((time.time()-startTime)/60, 3)))

NameError: name 'time' is not defined

In [ ]:
# band names from combined image
keys = ["FIRE_NAME"] + ee.Image(imageLst.get(0)).bandNames().getInfo()

In [ ]:
# Save as csv

# 167k fire points
saveSampleData(data=fireSampleData,
               keys=keys,
               geometry=firePts_df["geometry"],
               path="../../data/burned/sampleData.csv")

# 165k bbox points
saveSampleData(data=bboxSampleData,
               keys=keys,
               geometry=bboxPts_df["geometry"],
               path="../../data/unburned/sampleData.csv")

In [ ]:
# viz for every fire with post-fire Landsat 8 image, NLCD landcover, and classified burn severity as image layers

burnPalette = ["706c1e", "4e9d5c", "fff70b", "ff641b", "a41fd6"]

landCoverPalette = ["466b9f", "d1def8", "dec5c5", "d99282", "eb0000",
                    "ab0000", "b3ac9f", "68ab5f", "1c5f2c", "b5c58f",
                    "af963c", "ccb879", "dfdfc2", "d1d182", "a3cc51",
                    "82ba9e", "dcd939", "ab6c28", "b8d9eb", "6c9fb8"]


for name, date, geometry in bounds_df[["FIRE_NAME", "post-date", "geometry"]].values:
    geometry = ee.Geometry.Rectangle(list(geometry.bounds))
    center = geometry.centroid().getInfo()["coordinates"][::-1]

    fireImage = ee.Image(imageLst.filter(ee.Filter.eq("FIRE_NAME", name)).get(0)
                 ).clip(geometry)

    m = folium.Map(location=center, zoom_start=11.25)

    m.add_ee_layer(fireImage,
                   {"bands": ["SR_B7", "SR_B5", "SR_B3"], 
                    "gamma": [1.1, 1.1, 1],
                    "min": 1000, "max": 25000},
                    "Post Fire {}".format(date))

    m.add_ee_layer(fireImage, 
                  {"bands": ["landcover"],
                   "palette": landCoverPalette},
                   "Land Cover")
    
    m.add_ee_layer(fireImage, 
                  {"bands": ["burnSeverity"],
                   "min": 0, "max": 4,
                   "palette": burnPalette},
                   "Burn Severity")
    

    m.add_child(folium.LayerControl())
    print(name)
    display(m)    
    print("\n \n")